# Chi-Score Analysis Walkthrough:
In this notebook, each step of the chi-score analysis is executed as a separate cell. Each process is described in markup above the corresponding cell so that the user can easily access the data at different points in the analysis.

# Step 1: Installation
Skip this step if you already have the chi_score_analysis.py file downloaded to the working directory, otherwise execute the cell below to download the file.

In [ ]:
!pip install wget
import wget

wget.download('https://raw.githubusercontent.com/MWPlabUTSW/Chi-Score-Analysis/main/chi_score_analysis.py')

# Step 2: Import Analysis and Dependencies
Execute the following cell to import the packages that will be used in this notebook.

In [ ]:
import chi_score_analysis as xid
import matplotlib.pyplot as plt

# Step 3: Generate Pairwise Correlation Matrices
Enter the sequence to be analyzed as well as which window sizes to use in the analysis. Chi-Score matrices are stored in 'chi_matrices' and correlation matrices are stored in 'corr_matrices'. 

In this step, the input sequence is parsed into all possible subsequences of the specified length (window size) and the chi-score between each is calculated in a pairwise fashion. The chi-score matrices are then correlated and transformed into Pearson's Correlation Coefficients. 

The matrices can be visualized as matplotlib heatmaps. In the cell below, the matrices for 9 window sizes are computed, and the one generated using a window size of 12 is visualized.

In [ ]:
sequence = 'YOURSEQUENCE'
window_sizes = [6, 8, 10, 12, 14, 16, 18, 20, 22]

chi_matrices = list()
corr_matrices = list()
for window in window_sizes:
    chi_matrix = xid.get_heatmap_scores(sequence, window)
    chi_matrices.append(chi_matrix)
    corr_matrix = xid.get_corr_scores(chi_matrix)
    corr_matrices.append(corr_matrix)
    

hm = plt.imshow(corr_matrices[3], cmap = 'Blues')
plt.colorbar(hm)

# Step 4: Compute Insulation Scores
In order to identify an initial set of boundaries from the pairwise matrices, we calculate insulation score by moving a square window along the main diagonal of a matrix and computing the mean score contained within the polygon. Here, the insulation scores are high when the square is centered on a subsequence within an insulated region (the blue 'squares' along the diagonal) and low when centered on a subsequence between them. In this sense, the local minima of the insulation scores may represent boundaries between compositionally distinct regions. Execute the cell below to compute the insulation scores for all matrices in corr_matrices and plot the scores for the window size = 12 matrix. 

In [ ]:
insulation_scores = list()
for x in range(len(window_sizes)):
    insulation_scores.append(xid.get_insulation_scores(corr_matrices[x], s = (2*window_sizes[x])-1))

plt.plot(insulation_scores[3])

# Step 5: Getting Initial Set of Boundaries
The initial set of nodes from which boundary positions are optimized are determined from the insulation scores calculated in the previous step. Local minima from each set of insulation scores are first determined, resulting in a list of lists, each containing the potential boundary placements from one of the window sizes used in the analysis. The groups are then dissolved, and the boundary placements are reclustered based on their position on the sequence. For example, if the placements 50, 51, and 49 show up in three different window sizes, they will be grouped as the same boundary when reclustered.

In [ ]:
unclustered_groups = list()
for x in range(len(window_sizes)):
    unclustered_groups.append(xid.subsequence_to_boundary(xid.get_minima(insulation_scores[x]), window_sizes[x]))

clustered_groups = xid.cluster_boundaries(unclustered_groups)
unclustered_groups, clustered_groups

# Step 6: Optimizing Initial Boundaries
From the clustered groups generated in the previous step, an initial set of boundary placements can be generated by optizing the position of each group we just defined. This is done by selecting the position in each group that maximizes the chi-score between the produced modules. As part of the analysis we want to avoid significantly short modules; we do this here by merging any two consecutive boundary groups that result in a module shorter than the specified cutoff length. 

In [ ]:
optimized_solution = xid.optimize_boundaries(sequence, clustered_groups)
initial_solution, boundary_groups = xid.eliminate_short_modules(sequence, optimized_solution, clustered_groups, cutoff=6)
initial_solution, boundary_groups

# Step 7: Scoring and Trimming Boundaries
The final step of the analysis if to calculate z-scores for the optimized boundaries and iteratively remove low-scoring groups. The z-score for a boundary is calculated by taking the two regions separated by that boundary and randomly scrambling them together 500 times and the chi-score of the strongest boundary in each scramble is identified; from this set of boundary chi-scores, the z-score for the original boundary can be calculated. 

Once z-scores are calculated, the lowest-scoring boundary is removed and the remaining boundaries are reoptimized and rescored. This is done until only z-scores above the specified cutoff remain. The boundary positions and z-scores after each iteration are saved and output at the end. 

In [ ]:
initial_zscores = xid.get_zscores(sequence, initial_solution)
solutions, zscores = xid.trim_boundaries(sequence, initial_solution, initial_zscores, boundary_groups, cutoff=1.96)
solutions, zscores

# Step 8: Getting Modules and Plotting Solutions
The results of the previous cell can be used to get the resulting modules and/or plot them over the pairwise matrices for the sequence. Execute the following cell to plot a solution. By default, the window size = 12 matrix is plotted, however this can be changed as desired.

In [ ]:
modules = xid.get_modules(sequence, solutions[3])

xid.plot_solution(sequence, corr_matrices[3], solutions[3], window = 12, name = 'Input Sequence')
modules